In [1]:
import pandas as pd
import re

In [2]:
queries = pd.read_csv('/Users/nataliiaremezova/Documents/Workspace/BHT/BI/task_1/BHT_BI_WiSe2425/Task1/Data/QueriesToYT.csv')  # Replace with your file path
content_data = pd.read_csv('/Users/nataliiaremezova/Documents/Workspace/BHT/BI/task_1/BHT_BI_WiSe2425/Task1/Data/youtube_2006_month.csv')

In [3]:
# Preprocess text - lowercasing, stripping whitespace, and removing non-alphanumeric characters
def clean_text(text):
    if isinstance(text, str):
        text = text.lower().strip()
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    else:
        text = ''
    return text

queries['QUERY'] = queries['QUERY'].apply(clean_text)
content_data['title'] = content_data['title'].apply(clean_text)
content_data['tags'] = content_data['tags'].apply(lambda x: clean_text(str(x)))

# Check for matches between queries and video titles
queries['matched_titles'] = queries['QUERY'].apply(
    lambda q: content_data[content_data['title'].str.contains(q, na=False)].shape[0]
)

# Check for matches between queries and video tags
queries['matched_tags'] = queries['QUERY'].apply(
    lambda q: content_data[content_data['tags'].str.contains(q, na=False)].shape[0]
)

# Display results
print(queries[['QUERY', 'matched_titles', 'matched_tags']].head())

                            QUERY  matched_titles  matched_tags
0                     jake peyton               0             0
1             bush drinking video               0             0
2  cant stop scratching my vagina               0             0
3                      vid idiots               0             0
4                 naruto websites               0             0


In [5]:
# Filter queries with matched titles or tags greater than 0
matches = queries[(queries['matched_titles'] > 0) | (queries['matched_tags'] > 0)]

# Remove duplicate rows based on the 'QUERY' column
matches = matches.drop_duplicates(subset='QUERY')

# Display the filtered and deduplicated results
print(matches[['QUERY', 'matched_titles', 'matched_tags']])

                        QUERY  matched_titles  matched_tags
67                freerunning               0             2
72                       culo               1             1
131              the simpsons               1             1
134             myspace movie               0             1
171        brokeback mountain               1             2
200                   beatbox               4             5
250                      pedo               1             1
265              chuck norris               0             2
269                  my space               0             1
299   transformers theme song               0             1
379            something blue               1             0
578                  vogueing               0             1
602                    pacman               0             1
657                       tit              17            22
661         we are scientists               0             3
698                     naked           

In [11]:
matches.to_excel('../Data/matches.xlsx')

In [6]:
# Filter queries with matched titles or tags greater than 0
matches = queries[(queries['matched_titles'] > 0) | (queries['matched_tags'] > 0)]

# Add columns to display the matched titles and tags
matches['full_matched_titles'] = matches['QUERY'].apply(
    lambda q: content_data.loc[content_data['title'].str.contains(q, na=False), 'title'].tolist()
)

matches['full_matched_tags'] = matches['QUERY'].apply(
    lambda q: content_data.loc[content_data['tags'].str.contains(q, na=False), 'tags'].tolist()
)

# Remove duplicates based on 'QUERY'
matches = matches.drop_duplicates(subset='QUERY')

# Display the filtered and deduplicated results with matched titles and tags
print(matches[['QUERY', 'matched_titles', 'matched_tags', 'full_matched_titles', 'full_matched_tags']])


                        QUERY  matched_titles  matched_tags  \
67                freerunning               0             2   
72                       culo               1             1   
131              the simpsons               1             1   
134             myspace movie               0             1   
171        brokeback mountain               1             2   
200                   beatbox               4             5   
250                      pedo               1             1   
265              chuck norris               0             2   
269                  my space               0             1   
299   transformers theme song               0             1   
379            something blue               1             0   
578                  vogueing               0             1   
602                    pacman               0             1   
657                       tit              17            22   
661         we are scientists               0          

/var/folders/7r/8tzl4xz922146pgx3szt94t80000gn/T/ipykernel_60661/2761361353.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['full_matched_titles'] = matches['QUERY'].apply(
/var/folders/7r/8tzl4xz922146pgx3szt94t80000gn/T/ipykernel_60661/2761361353.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['full_matched_tags'] = matches['QUERY'].apply(


In [7]:
# Categories from your input
categories = {
    "Film & Animation", "Autos & Vehicles", "Music", "Pets & Animals", "Sports", 
    "Short Movies", "Travel & Events", "Gaming", "Videoblogging", "People & Blogs", 
    "Comedy", "Entertainment", "News & Politics", "Howto & Style", "Education", 
    "Science & Technology", "Nonprofits & Activism", "Movies", "Anime/Animation", 
    "Action/Adventure", "Classics", "Documentary", "Drama", "Family", "Foreign", 
    "Horror", "Sci-Fi/Fantasy", "Thriller", "Shorts", "Shows", "Trailers"
}

# Assign videos to categories based on matching tags
def categorize_tags(tags):
    if pd.isna(tags):
        return 'Uncategorized'
    tags = tags.lower()
    for category in categories:
        if category.lower() in tags:
            return category
    return 'Uncategorized'

content_data['category'] = content_data['tags'].apply(categorize_tags)

# Create hierarchy with a general category, specific category, and tags
general_categories = {
    'Education': ['Education', 'Science & Technology', 'Howto & Style'],
    'Music': ['Music'],
    'Movies': ['Film & Animation', 'Movies', 'Short Movies', 'Documentary', 'Drama'],
    'Entertainment': ['Entertainment', 'Comedy', 'People & Blogs', 'Shows'],
    'News & Politics': ['News & Politics'],
    'Gaming': ['Gaming'],
    'Travel': ['Travel & Events'],
    # Add more general categories and mappings as needed
}

def assign_general_category(category):
    for general, specifics in general_categories.items():
        if category in specifics:
            return general
    return 'Other'

content_data['general_category'] = content_data['category'].apply(assign_general_category)

# Display the hierarchy
hierarchy = content_data[['general_category', 'category', 'tags']].head()
print(hierarchy)


  general_category       category  \
0            Other  Uncategorized   
1            Other  Uncategorized   
2            Other  Uncategorized   
3            Music          Music   
4            Music          Music   

                                                tags  
0  mad world gary jules donnie darko michel gondr...  
1                       mexico banda antonio aguilar  
2  hilltop hoods nosebleed section simon brien fl...  
3  sisters mercy eldritch lucretia reflection 80s...  
4        story year anthem our dying day music video  


In [9]:
# Filter out 'Uncategorized' categories
categorized_videos = content_data[content_data['category'] != 'Uncategorized']
print(categorized_videos)

# Display the unique categories that are not 'Uncategorized'
print(categorized_videos['category'].unique())


               publishedAt                 channelId  \
3     2006-01-24T21:13:53Z  UCPtUzr5y-aMmdfWurqNpHOg   
4     2006-01-11T02:02:19Z  UCiPXAzD0HnGDHRCwleX2I2A   
8     2006-01-25T19:19:21Z  UCY30JRSgfhYXA6i6xX1erWg   
9     2006-01-14T01:44:43Z  UCCIKHL_vC_OAKLphvoXpNdg   
16    2006-01-24T21:19:27Z  UCPtUzr5y-aMmdfWurqNpHOg   
...                    ...                       ...   
2580  2006-12-16T20:22:09Z  UCeG9ubc2KnEbQYGlIl45swQ   
2601  2006-12-10T02:07:03Z  UC8pdWMu8TsTMrn5ngIjyD2w   
2603  2006-12-17T21:09:42Z  UCXPBWg-9k6_-_SPtPYnpTqA   
2611  2006-12-09T22:00:54Z  UCXPBWg-9k6_-_SPtPYnpTqA   
2612  2006-12-09T23:51:27Z  UCXPBWg-9k6_-_SPtPYnpTqA   

                                             title  \
3     the sisters of mercy  lucretia my reflection   
4         story of the yearanthem of our dying day   
8               teenage mutant ninja turtles theme   
9       allison crowe  hallelujah  liveinthestudio   
16                      the sisters of mercy  more   
...

In [ ]:
categorized_videos.to_excel('../Data/categorized_videos.xlsx')